In [8]:
from flightanalysis.fc_json import FCJson
from flightplotting.plots import plotdtw, plotsec
from flightanalysis import Section

with open("FlightAnalysis/tests/test_inputs/manual_F3A_P21_21_09_24_00000052.json", "r") as f:
    fcj = FCJson.parse_fc_json(f)

#sec = Section.from_csv("FlightAnalysis/tests/test_inputs/test_log_00000052_section.csv")

#sec = Section(sec.data.loc[sec.data.z >10])
#sec.data.index = sec.data.index - sec.data.index[0]


/home/tom/miniconda3/envs/pyfc/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/tom/miniconda3/envs/pyfc/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/tom/source/PyFlightCoach/FlightAnalysis/flightanalysis/section.py:390: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



In [9]:
import plotly.graph_objects as go
from geometry import Points
fig=go.Figure()

fig.add_trace(go.Scatter(x=fcj.sec.data.index, y=fcj.sec.brvr, name="json"))

fig.show()


In [5]:
from flightanalysis import get_schedule
from flightanalysis.schedule.element import get_rates

rates = get_rates(fcj.sec)


sched = get_schedule("F3A", "P21").scale_distance(rates["distance"])

rates

{flightanalysis.schedule.element.Loop: 1.0624753749298652,
 flightanalysis.schedule.element.Line: 4.1796307755770625,
 flightanalysis.schedule.element.Snap: 22.477170042983705,
 flightanalysis.schedule.element.StallTurn: 3.728727210546274,
 flightanalysis.schedule.element.Spin: 13.563076241845026,
 'speed': 29.269122645586474,
 'distance': 163.36709286341568}

In [ ]:
from flightanalysis import Section
# this is needed to get just the scored sequence from a section
sequence = fcj.sec.subset(sched.manoeuvres[0].get_data(fcj.sec).data.index[0],sched.manoeuvres[-2].get_data(fcj.sec).data.index[-1]).remove_labels()

dist, aligned = Section.align(sequence, sched.create_raw_template(sequence.get_state_from_index(0).direction, 1.5*rates["speed"], rates["distance"]), 2)
dist


In [ ]:
sum(fcj._dists)

In [ ]:
from plotly.subplots import make_subplots
import flightplotting.templates
from flightplotting.traces import dtwtrace
import numpy as np

man = sched.manoeuvres[1]

fig = make_subplots(
    1,
    2,
    specs=[[{'is_3d': True}, {'is_3d': True}]]
)
fig.update_layout(template="flight3d", showlegend=False, width=1200, height=800)


fig.add_traces(
    dtwtrace(man.get_data(aligned), man.elements), 
    rows=list(np.full(len(man.elements)+2, 1)),
    cols=list(np.full(len(man.elements)+2, 1))
)
fig.add_traces(
    dtwtrace(man.get_data(fcj.sec), man.elements),  
    rows=list(np.full(len(man.elements)+2, 1)),
    cols=list(np.full(len(man.elements)+2, 2))
)
fig.show()

